In [3]:
# import requisite libraries
import pandas as pd
import numpy as np
import os
import cv2
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Rescaling, Resizing
from PIL import Image
import csv
import random
import shutil


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
%cd /content/drive/MyDrive/Capstone/


/content/drive/MyDrive/Capstone


In [7]:
"#ensure file is a ".png" image and import it.
dir_path = '/content/drive/My Drive/Capstone/pictures_grouped'
for filename in os.listdir(dir_path):
    if filename.endswith(".png"): # check if file is an image
        img_path = os.path.join(dir_path, filename)
        img = Image.open(img_path)


In [8]:
print(os.getcwd())


/content/drive/My Drive/Capstone


In [9]:
# Count the number of image sin each subdiretory this way you know how many you need to add to each class to balance
your_directory = 'pictures_grouped'

# iterate over all the directories and subdirectories
for dirpath, dirnames, filenames in os.walk(your_directory):
    # count .png files
    images = [fn for fn in filenames if fn.endswith(('.png'))]
    print(f'There are {len(images)} images in {dirpath}')


There are 0 images in pictures_grouped
There are 1615 images in pictures_grouped/6_to_18
There are 1111 images in pictures_grouped/65_and_older
There are 900 images in pictures_grouped/18_to_24
There are 910 images in pictures_grouped/45_to_54
There are 858 images in pictures_grouped/35_to_44
There are 1336 images in pictures_grouped/25_to_34
There are 738 images in pictures_grouped/55_to_64
There are 2312 images in pictures_grouped/5_and_Under


In [10]:
def balance_class(base_directory, mvp_rotated_dir, class_dir, additional_images):
    # Get the count of images in the directory
    count = len([name for name in os.listdir(os.path.join(base_directory, class_dir)) if name.endswith('.png')])

    directory = os.path.join(base_directory, class_dir)
    new_directory = os.path.join(mvp_rotated_dir, class_dir)

    # Create the new directory if it doesn't exist
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)

    # Get a list of all .png images in the directory
    images = [fn for fn in os.listdir(directory) if fn.endswith('.png')]

    # Copy all original images to the new directory
    for picture in images:
        shutil.copy(os.path.join(directory, picture), os.path.join(new_directory, picture))

    # Randomly select images to rotate until we reach the desired count
    for i in range(additional_images):
        image_name = random.choice(images)

        # Open the image file
        img = Image.open(os.path.join(directory, image_name))

        # Rotate the image by a random degree between 0 and 90
        rotation_degree = random.uniform(0, 90)
        img_rotated = img.rotate(rotation_degree)

        # Save the rotated image with a new name in the new directory
        base_name, ext = os.path.splitext(image_name)
        new_image_name = f'{base_name}_rotated_{count+i}{ext}'
        img_rotated.save(os.path.join(new_directory, new_image_name))

# Your main directory and new base directory
base_directory = '/content/drive/My Drive/Capstone/pictures_grouped'
mvp_rotated_dir = '/content/drive/My Drive/Capstone/rotated_images_mvp'



In [11]:
# Call the function for each class with the desired number of additional images
balance_class(base_directory, mvp_rotated_dir, '5_and_Under', 100)
balance_class(base_directory, mvp_rotated_dir, '6_to_18', 600)
balance_class(base_directory, mvp_rotated_dir, '18_to_24', 900)
balance_class(base_directory, mvp_rotated_dir, '25_to_34', 900)
balance_class(base_directory, mvp_rotated_dir, '35_to_44', 858)
balance_class(base_directory, mvp_rotated_dir, '45_to_54', 910)
balance_class(base_directory, mvp_rotated_dir, '55_to_64', 738)
balance_class(base_directory, mvp_rotated_dir, '65_and_older', 1000)



In [12]:
#COUNTS THE NUMBER OF IMAGES IN EACH DIRECTORY TO CONFIRM NUMBER OF IMAGES IN EACH DIRECTORY
def count_images_in_subdirs(parent_dir):
    # Get a list of all subdirectories in the parent directory
    subdirs = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]

    for subdir in subdirs:
        subdir_path = os.path.join(parent_dir, subdir)
        # Get a list of all image files in the subdirectory
        image_files = [f for f in os.listdir(subdir_path) if f.endswith(('.png'))]
        # Print the number of images
        print(f"{subdir}: {len(image_files)} images")

count_images_in_subdirs('/content/drive/MyDrive/Capstone/rotated_images_mvp')


5_and_Under: 2412 images
6_to_18: 2215 images
18_to_24: 1800 images
25_to_34: 2236 images
35_to_44: 1716 images
45_to_54: 1820 images
55_to_64: 1476 images
65_and_older: 2111 images


In [13]:
def flip_images_in_folder(base_directory, rotated_flipped_mvp, num_images_to_flip_dict):
    # Get a list of all subdirectories in the folder
    subdirs = [d for d in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, d))]

    for subdir in subdirs:
        subdir_path = os.path.join(base_directory, subdir)
        output_subdir_path = os.path.join(rotated_flipped_mvp, subdir)
        # Create the output subdirectory if it doesn't exist
        if not os.path.exists(output_subdir_path):
            os.makedirs(output_subdir_path)
        # Get a list of all image file names in the subdirectory
        image_files = os.listdir(subdir_path)

        # Copy all images to the output directory
        for img_file in image_files:
            shutil.copy(os.path.join(subdir_path, img_file), os.path.join(output_subdir_path, img_file))

        # Randomly select a subset of images to flip
        num_images_to_flip = num_images_to_flip_dict.get(subdir, 0)  # Use a default of 0 if the subdir is not in the dict
        images_to_flip = random.sample(image_files, min(num_images_to_flip, len(image_files)))  # Don't try to flip more images than exist

        for img_file in images_to_flip:
            try:
                # Read the image file
                img = tf.keras.preprocessing.image.load_img(os.path.join(subdir_path, img_file))
                # Convert PIL Image to Numpy array
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                # Flip the image horizontally
                flipped_img = tf.image.flip_left_right(img_array)
                # Convert Numpy array to PIL Image
                flipped_img_pil = tf.keras.preprocessing.image.array_to_img(flipped_img)
                # Save the flipped image to the output directory with '_flipped' added to the end of the filename
                flipped_img_pil.save(os.path.join(output_subdir_path, os.path.splitext(img_file)[0] + '_flipped' + os.path.splitext(img_file)[1]))
            except Exception as e:
                print(f"Error processing file {img_file}: {e}")

In [17]:
# Define your number of flips for each subdirectory
num_images_to_flip_dict = {
    '5_and_Under': 200,
    '6_to_18': 397,
    '18_to_24': 812,
    '25_to_34': 376,
    '35_to_44': 896,
    '45_to_54': 792,
    '55_to_64': 1136,
    '65_and_older': 501
}

base_directory= '/content/drive/MyDrive/Capstone/rotated_images_mvp'
rotated_flipped_mvp= '/content/drive/MyDrive/Capstone/rotated_flipped_mvp'

flip_images_in_folder(base_directory, rotated_flipped_mvp, num_images_to_flip_dict)

In [ ]:
#COUNTS THE NUMBER OF IMAGES IN EACH DIRECTORY TO CONFIRM CLASS BALANCE
def count_images_in_subdirs(parent_dir):
    # Get a list of all subdirectories in the parent directory
    subdirs = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]

    for subdir in subdirs:
        subdir_path = os.path.join(parent_dir, subdir)
        # Get a list of all image files in the subdirectory
        image_files = [f for f in os.listdir(subdir_path) if f.endswith(('.png'))]
        # Print the number of images
        print(f"{subdir}: {len(image_files)} images")

count_images_in_subdirs('/content/drive/MyDrive/Capstone/rotated_flipped_mvp')


5_and_Under: 2612 images
6_to_18: 2612 images
18_to_24: 2612 images
25_to_34: 2612 images
35_to_44: 2612 images
45_to_54: 2612 images
55_to_64: 2612 images
65_and_older: 2612 images
